Подключение библиотек. YOLO (You Only Look Once) - это алгоритм машинного обучения, который используется для обнаружения объектов на изображениях и видео.

In [7]:
from ultralytics import YOLO
import numpy as np
import cv2

Загрузка модели и детекция

In [10]:
# Загружаем модель YOLOv8x
model = YOLO("yolov8x.pt")

# Загружаем видео
video = cv2.VideoCapture("test_video.mp4")

# Инициализация счетчика ID для трекинга
frame_count = 0
track_ids = {}
   

In [11]:
# Цикл по кадрам видео
while(video.isOpened()):
    # Читаем кадр
    ret, frame = video.read()
    if not ret:
        break

    # Выполняем детекцию
    results = model.predict(frame, classes=[0], conf=0.2, iou=0.7)

    # Получаем координаты распознанных людей
    detections = results[0].boxes.xyxy.cpu().numpy().astype(int)

    # Обновляем словарь track_ids
    for det in detections:
        x1, y1, x2, y2 = det
        center_x = (x1 + x2) // 2
        center_y = (y1 + y2) // 2

        # Проверяем, есть ли уже ID для этого человека
        found_id = False
        for id, (prev_x, prev_y) in track_ids.items():
            # Проверяем расстояние между центрами
            distance = np.sqrt((center_x - prev_x)**2 + (center_y - prev_y)**2)
            if distance < 50:
                track_ids[id] = (center_x, center_y)
                found_id = True
                break

        # Если ID не найден, создаем новый
        if not found_id:
            new_id = len(track_ids) + 1
            track_ids[new_id] = (center_x, center_y)

    # Рисуем прямоугольники и ID
    for id, (center_x, center_y) in track_ids.items():
        # Находим соответствующую детекцию
        for det in detections:
            x1, y1, x2, y2 = det
            if (x1 < center_x < x2) and (y1 < center_y < y2):
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f"ID: {id}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Выводим кадр
    cv2.imshow("Video", frame)

    # Увеличиваем счетчик кадров
    frame_count += 1

    # Выход из цикла при нажатии клавиши 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Освобождаем ресурсы
cap.release()
cv2.destroyAllWindows()


0: 384x640 7 persons, 1765.9ms
Speed: 2.0ms preprocess, 1765.9ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 1757.6ms
Speed: 2.0ms preprocess, 1757.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 1734.6ms
Speed: 2.0ms preprocess, 1734.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 persons, 1742.1ms
Speed: 2.0ms preprocess, 1742.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 1745.6ms
Speed: 2.0ms preprocess, 1745.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 1732.8ms
Speed: 2.0ms preprocess, 1732.8ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 1736.6ms
Speed: 2.0ms preprocess, 1736.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 1731.6ms
Speed: 2.0ms preprocess, 1731.6ms inference, 2.0ms 